## 매 시도마다 돌릴 것들

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from tqdm import tqdm
from collections import Counter
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoConfig
from scipy.special import softmax
import shutil
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import seaborn as sns
import torch.nn as nn
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 전처리

In [2]:
# 텍스트 데이터 로드
texts_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/texts.csv', encoding='unicode_escape')
label_txt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt', sep='\t', header=0)
label_txt.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']
label_jpg = label_txt.copy()

def pre_text(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

texts_df['Text'] = texts_df['Text'].apply(pre_text)

In [3]:
dic = {'positive': 1, 'neutral':0, 'negative': -1}
def pre_labeltext(text):
    new_text = text.split(",")[0]
    return dic[new_text]
label_txt['Annotator1'] = label_txt['Annotator1'].apply(pre_labeltext)
label_txt['Annotator2'] = label_txt['Annotator2'].apply(pre_labeltext)
label_txt['Annotator3'] = label_txt['Annotator3'].apply(pre_labeltext)
label_txt['label'] = label_txt['Annotator1'] + label_txt['Annotator2'] + label_txt['Annotator3']
label_txt['label'] = label_txt['label']/3
label_txt = label_txt.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [4]:
def pre_labeltext(text):
    new_text = text.split(",")[1]
    return dic[new_text]
label_jpg['Annotator1'] = label_jpg['Annotator1'].apply(pre_labeltext)
label_jpg['Annotator2'] = label_jpg['Annotator2'].apply(pre_labeltext)
label_jpg['Annotator3'] = label_jpg['Annotator3'].apply(pre_labeltext)
label_jpg['label'] = label_jpg['Annotator1'] + label_jpg['Annotator2'] + label_jpg['Annotator3']
label_jpg['label'] = label_jpg['label']/3
label_jpg = label_jpg.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [5]:
# 라벨 데이터와 텍스트 데이터 병합
merged_text = pd.merge(label_txt, texts_df, on='ID')

## 데이터 라벨링-모델 일치율 확인(만족스럽다)

In [6]:
def checksentiment(listtexts, listlabels):
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    batch_size = 16
    sentiment_scores = []

    for i in tqdm(range(0, len(listtexts), batch_size)):
        batch_texts = listtexts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
        encoded_input.to(device)

        with torch.no_grad():
            outputs = model(**encoded_input)

        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        scores = probabilities.cpu().numpy()

        pos_probs = scores[:, 2]  # 긍정 확률
        neg_probs = scores[:, 0]  # 부정 확률
        sentiments = pos_probs - neg_probs  # 감성 점수
        sentiment_scores.extend(sentiments)

    listlabels = np.array(listlabels)
    sentiment_scores = np.array(sentiment_scores)

    # L2 Loss
    l2_loss = np.mean((sentiment_scores - listlabels) ** 2)
    print(f"L2 Loss: {l2_loss}")

    return sentiment_scores

In [7]:
lab = merged_text['label'].tolist()
sentiment_scores = checksentiment(merged_text['Text'].tolist(), lab)
check = 0
for i in range(len(sentiment_scores)):
    if sentiment_scores[i] * lab[i] < 0:
        check+=1
check

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT e

L2 Loss: 0.22856436414728568


1713

## 이미지 설명-감성 분석 점수 확인(학습 전 저성능 확인)

In [10]:
#자기 전에 켜 둘 것
source_dir = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data'

target_dir = '/content/data'

# 대상 폴더가 없으면 생성
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

loader = label_txt['ID'].tolist()

for filename in tqdm(loader):
    filename_with_ext = f"{filename}.jpg"
    src_file = os.path.join(source_dir, filename_with_ext)
    dst_file = os.path.join(target_dir, filename_with_ext)

    try:
        shutil.copy2(src_file, dst_file)
    except Exception as e:
        print(f"파일 복사 중 오류 발생: {filename_with_ext}, 오류: {e}")

100%|██████████| 19598/19598 [4:00:47<00:00,  1.36it/s]


In [8]:
merged = pd.merge(merged_text, label_jpg, on='ID')

In [9]:
merged = pd.merge(merged, label_txt, on='ID')

In [10]:
scores = np.array(sentiment_scores)
sim = merged['label_x'] * merged['label_y'] > 0
merged['pred'] = scores
input_df = merged[sim]

In [11]:
input_df.drop([1336], axis=0, inplace=True)

<ipython-input-11-a5e798a59b6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df.drop([1336], axis=0, inplace=True)


In [12]:
image_dir = '/content/data'
def jpg(num):
    return f"/content/data/{num}.jpg"
input_df['ID'] = input_df['ID'].apply(jpg)
image_ids = input_df['ID'].tolist()
actual_labels = input_df['pred'].tolist()
actual_text = input_df['Text'].tolist()

<ipython-input-12-a118077d8fca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['ID'] = input_df['ID'].apply(jpg)


## 테스트

In [13]:
!pip install datasets pandas

In [19]:
from datasets import Dataset

In [20]:
dataset = Dataset.from_pandas(input_df[['ID', 'Text']])

In [21]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = Image.open(item["ID"]).convert('RGB')
        encoding = self.processor(images=image, text=item["Text"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [22]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [23]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=False, batch_size=4)

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.to(device)

model.train()

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [26]:
model.train()
for epoch in range(10):
    totloss = 0
    print("Epoch:", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader)):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)

        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss
        totloss += loss

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
    print('train loss', totloss)

Epoch: 0


0it [00:00, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
740it [07:59,  1.54it/s]


KeyboardInterrupt: 

## 테스트

In [76]:
labels = actual_labels
texts = actual_text
image_dir = '/content/data'

In [79]:
train_image_ids = image_ids
train_labels = actual_labels

In [80]:
# 감성 분석 모델 로드 (고정된 상태로 사용)
MODEL_SA = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer_sa = AutoTokenizer.from_pretrained(MODEL_SA)
config_sa = AutoConfig.from_pretrained(MODEL_SA)
model_sa = AutoModelForSequenceClassification.from_pretrained(MODEL_SA)
model_sa.to(device)
model_sa.eval()  # 평가 모드

# 감성 분석 모델의 파라미터를 고정
for param in model_sa.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
# 손실 함수 정의 (예: MSELoss)
criterion = nn.MSELoss()

# 옵티마이저 정의 (BLIP 모델의 파라미터만 업데이트)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

In [82]:
num_epochs = 100  # 최대 에포크 수를 크게 설정
patience = 5      # 검증 손실이 개선되지 않는 에포크 수 허용치
best_val_loss = float('inf')
epochs_no_improve = 0
early_stop = False

In [122]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, image_ids, labels, image_dir, processor):
        self.image_ids = image_ids
        self.labels = labels
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_ids[idx])
        image = Image.open(image_path).convert('RGB')
        # 이미지 전처리 수행
        pix = self.processor(image, return_tensors='pt')
        # 배치 차원 제거 (processor는 보통 배치 차원을 포함하여 반환)
        pix = {k: v.squeeze(0) for k, v in pix.items()}
        label = self.labels[idx]
        return pix, label

# 데이터셋과 데이터로더 생성
train_dataset1 = ImageDataset(train_image_ids, train_labels, image_dir, processor)

batch_size = 32  # 적절한 배치 사이즈 설정
train_dataloader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=False)


In [68]:
model.eval()
total_loss = 0.0
for batch in tqdm(train_dataloader):
    pix, labels = batch
    pix = {k: v.to(device) for k, v in pix.items()}
    labels = labels.to(device).float()

    # BLIP 모델을 통해 캡션 생성
    outputs = model.generate(**pix)
    # 생성된 토큰 ID를 디코딩하여 텍스트로 변환
    captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

    # 감성 분석 모델을 통해 감성 점수 계산
    inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs_sa = model_sa(**inputs_sa)
    scores = outputs_sa.logits.cpu().numpy()
    probs = softmax(scores, axis=1)
    pos_probs = probs[:, 2]  # 긍정 확률 (모델에 따라 인덱스 확인 필요)
    neg_probs = probs[:, 0]  # 부정 확률
    sentiment_scores = pos_probs - neg_probs  # [batch_size]

    # 감성 점수를 텐서로 변환
    sentiment_scores = torch.from_numpy(sentiment_scores).float().to(device)

    # 손실 계산 (예: MSELoss 사용)
    loss = criterion(sentiment_scores, labels)

    total_loss += loss.item()

avg_loss = total_loss / len(train_dataloader)
print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

  0%|          | 0/402 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 44%|████▍     | 178/402 [12:50<16:10,  4.33s/it]


KeyboardInterrupt: 